# FinViz Scrape for Data Augmentation
* Goal: Pull additional data on tickers using finviz such as market cap

In [3]:
! pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 8.1 MB/s eta 0:00:0000:0100:01
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pandas.io.common
import csv
from csv import DictWriter, writer
from pprint import pprint

import time
from datetime import datetime, timedelta

URL = "https://finviz.com/"
ticker = "META"
tickers = ["AAPL", "SPY", "CAT", "WMT", 'sadfvsebewrtb']

In [2]:
#pull SP500 tickers
sp500_tickers = pd.read_csv("SPY_companies.csv")
sp500_tickers = sp500_tickers.Symbol.tolist()

# pull larger list of all tickers (5783 total) for more data
tickers_all = pd.read_csv("stock_info.csv")
tickers_all = tickers_all.Ticker.unique().tolist()

tickers_all = list(set(tickers_all+sp500_tickers))
print(len(tickers_all))


5867


In [3]:
filename = "/Users/austinpaxton/Documents/ML_Bootcamp/Capstone/data/stock_fundamentals.csv"

#initialize chrome driver
chrome_driver_path = "/Applications/Drivers/chromedriver"
driver = webdriver.Chrome(executable_path=chrome_driver_path)

#get finviz stock page
driver.get(URL)
# #initialize list to store lists for each ticker
# data=[]

tickers = tickers_all


for ticker in tickers:
    try:
        print(f"\n ------------Searching for ticker: {ticker} || {round((tickers.index(ticker)+1)/len(tickers)*100,3)}% Complete-----------")
        search_bar2 = driver.find_element(By.CSS_SELECTOR, "td#search input")
        search_bar2.send_keys(ticker)
        search_button2 = driver.find_element(By.CSS_SELECTOR,"td#search span")
        search_button2.click()

        #get stock meta data      
        title_table = driver.find_element(By.CSS_SELECTOR, "table.fullview-title")
        ticker = title_table.find_element(By.CSS_SELECTOR, "td>a#ticker").text
        full_name = title_table.find_element(By.CSS_SELECTOR,  "tr>td>a.tab-link>b").text
        last_row = title_table.find_elements(By.CSS_SELECTOR, "tr>td.fullview-links a")
        sector= last_row[0].text
        industry = last_row[1].text
        country = last_row[2].text
        meta_data = [ticker, full_name, sector, industry, country]
        meta_names = ["ticker", "full_name", "sector", "industry", "country"]  
        print("-Meta Data Obtained-")

        # get fundamental data
        driver.execute_script("window.scrollTo(0, 700)") #Short scroll
        table= driver.find_element(By.CSS_SELECTOR, "table.snapshot-table2")
        cell_names_elements = table.find_elements(By.CSS_SELECTOR,"td.snapshot-td2-cp")
        cell_names = [element.text for element in cell_names_elements]
        cell_values_elements = table.find_elements(By.CSS_SELECTOR,"td.snapshot-td2")
        cell_data = [element.text for element in cell_values_elements]
        print("-Fundamentals Data Obtained-")
            
        #combine meta and fundamental column names into a list
        headers = meta_names + cell_names
        #combine meta and fundamental data
        stock_data = meta_data+cell_data
        print("-Meta and Fundamental Data Successfully Merged-")
        #convert to dict to add to json file:
        new_entry = dict(zip(headers,stock_data))
        #pprint(new_entry)

        if tickers.index(ticker) ==0: #check if csv file has headers on first ticker
            try: 
                csv_contents =pd.read_csv(filename)
            except pandas.errors.EmptyDataError: #if empty csv add headers
                with open(filename, "a") as csv_file: #write new object to file
                    header_object = writer(csv_file)
                    header_object.writerow(headers)
                    csv_file.close()
        
            
        
        with open(filename, "a") as csv_file: #write new object to file
                dict_object = DictWriter(csv_file, fieldnames=dict_keys)
                dict_object.writerow(new_entry)
                
        print("-Successfully Appended Data to csv file-")
        #note: we append to csv instead of json bc json requires downloading the entire json object and appending and then rewrittingon each iteration

    except NoSuchElementException:
        #add "-" if ticker not found
        print("no such element exception")

    except ElementClickInterceptedException: #for ad interference
        try:
            #clear pop-up
            driver.find_element(By.CSS_SELECTOR,"div#modal-elite-ad_content button").click()
        except:
            print("could not clear pop up ")
        driver.back()  #goes back to previous page
        print("back to previous page")

    except StaleElementReferenceException:
        print("Stale Element Exception")
        driver.back()  #goes back to previous page
        print("back to previous page")

    except:
        print("un idnetified error")
        driver.back()  #goes back to previous page
        print("back to previous page")

        

time.sleep(10000)
driver.quit()

/var/folders/hn/vdq8ffhj0zq9blr74w80pr180000gn/T/ipykernel_1471/2408347824.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)



 ------------Searching for ticker: CAT || 0.017% Complete-----------
-Meta Data Obtained-
-Fundamentals Data Obtained-
-Meta and Fundamental Data Successfully Merged-
un idnetified error
back to previous page

 ------------Searching for ticker: MYF || 0.034% Complete-----------
no such element exception

 ------------Searching for ticker: ON || 0.051% Complete-----------
-Meta Data Obtained-
-Fundamentals Data Obtained-
-Meta and Fundamental Data Successfully Merged-
un idnetified error
back to previous page

 ------------Searching for ticker: CHKR || 0.068% Complete-----------
no such element exception

 ------------Searching for ticker: NMK-C || 0.085% Complete-----------
no such element exception

 ------------Searching for ticker: SLB || 0.102% Complete-----------
-Meta Data Obtained-
-Fundamentals Data Obtained-
-Meta and Fundamental Data Successfully Merged-
un idnetified error
back to previous page

 ------------Searching for ticker: PTP || 0.119% Complete-----------
Stale Elem

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x000000010120eae8 chromedriver + 4958952
1   chromedriver                        0x00000001012067b3 chromedriver + 4925363
2   chromedriver                        0x0000000100dc81d6 chromedriver + 475606
3   chromedriver                        0x0000000100d9dcec chromedriver + 302316
4   chromedriver                        0x0000000100e36f2f chromedriver + 929583
5   chromedriver                        0x0000000100e4c926 chromedriver + 1018150
6   chromedriver                        0x0000000100e319e3 chromedriver + 907747
7   chromedriver                        0x0000000100dfbaef chromedriver + 686831
8   chromedriver                        0x0000000100dfd00e chromedriver + 692238
9   chromedriver                        0x00000001011ca9ad chromedriver + 4680109
10  chromedriver  

In [14]:
# add header to stock_fundamentals df
df = pd.read_csv("data/stock_fundamentals.csv",header=None)
df.columns = headers
df.to_csv("data/finviz_4-12-23.csv",index=False)


ValueError: Length mismatch: Expected axis has 78 elements, new values have 77 elements